In [1]:
!nvidia-smi

Tue Mar 26 10:29:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
tf.keras.applications.inception_resnet_v2.preprocess_input

<function keras.src.applications.inception_resnet_v2.preprocess_input(x, data_format=None)>

In [4]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=6,
    classifier_activation="softmax",
)

219055592/219055592 [==============================] - 1s 0us/step


In [5]:
dataset_train = '/content/drive/MyDrive/garbage_dataset/garbage_dataset/train'
dataset_test = '/content/drive/MyDrive/garbage_dataset/garbage_dataset/test'

In [6]:
import os

# Function to count files in a directory
def count_files(directory):
    return len([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])

# Iterate over each folder and count the number of images
image_counts = {}
for folder in os.listdir(dataset_train):
    folder_path = os.path.join(dataset_train, folder)
    if os.path.isdir(folder_path):
        image_counts[folder] = count_files(folder_path)

# Print the results
print("No. of images in each folder for training-->")
total_count = 0
for folder, count in image_counts.items():
    print(f"{folder}: {count} images")
    total_count += count
print('Length of training dataset: {}' .format(total_count))

# Function to count files in a directory
def count_files(directory):
    return len([filename for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])

# Iterate over each folder and count the number of images
image_counts = {}
for folder in os.listdir(dataset_test):
    folder_path = os.path.join(dataset_test, folder)
    if os.path.isdir(folder_path):
        image_counts[folder] = count_files(folder_path)

# Print the results
print("No. of images in each folder for testing-->")
total_count = 0
for folder, count in image_counts.items():
    print(f"{folder}: {count} images")
    total_count += count
print('Length of test dataset: {}' .format(total_count))

No. of images in each folder for training-->
trash: 91 images
glass: 354 images
plastic: 347 images
metal: 286 images
paper: 403 images
cardboard: 287 images
Length of training dataset: 1768
No. of images in each folder for testing-->
paper: 108 images
trash: 29 images
metal: 68 images
plastic: 74 images
glass: 82 images
cardboard: 70 images
Length of test dataset: 431


In [7]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

In [9]:
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
from keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator for training data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.2, 0.5],
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

# ImageDataGenerator for test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical',
    shuffle=True
)

# Flow validation images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
    dataset_test,
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical',
    shuffle=False
)


Found 1768 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


In [13]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [14]:
import subprocess

# Execute shell command to remove the ./logs/ directory
subprocess.run(["rm", "-rf", "./logs/"])

CompletedProcess(args=['rm', '-rf', './logs/'], returncode=0)

In [15]:
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

start = datetime.now()

# Define TensorBoard callback
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Define ModelCheckpoint callback to save the best model based on training loss and accuracy
checkpoint_callback = ModelCheckpoint(filepath="./best_model.keras",
                                      monitor='accuracy',  # Monitor training loss
                                      save_best_only=True,  # Save the best model
                                      mode='max',  # Save the model with minimum loss
                                      verbose=1)  # Show progress

history = model.fit(
    train_generator,
    epochs=100,
    steps_per_epoch=10,
    validation_data=test_generator,
    callbacks=[tensorboard_callback, checkpoint_callback]
)

duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/100
10/10 [==============================] - ETA: 0s - loss: 7.5049 - accuracy: 0.1800
Epoch 1: accuracy improved from -inf to 0.18000, saving model to ./best_model.keras
10/10 [==============================] - 130s 13s/step - loss: 7.5049 - accuracy: 0.1800 - val_loss: 2.9648 - val_accuracy: 0.3991
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 2.0893 - accuracy: 0.4000
Epoch 2: accuracy improved from 0.18000 to 0.40000, saving model to ./best_model.keras
10/10 [==============================] - 33s 4s/step - loss: 2.0893 - accuracy: 0.4000 - val_loss: 1.9004 - val_accuracy: 0.4826
Epoch 3/100
10/10 [==============================] - ETA: 0s - loss: 2.0945 - accuracy: 0.4200
Epoch 3: accuracy improved from 0.40000 to 0.42000, saving model to ./best_model.keras
10/10 [==============================] - 32s 3s/step - loss: 2.0945 - accuracy: 0.4200 - val_loss: 2.8972 - val_accuracy: 0.3480
Epoch 4/100
10/10 [==============================] - ETA: 0s - loss:

In [ ]:
%tensorboard --logdir logs/fit